In [3]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext import data, datasets, vocab

import numpy as np

from argparse import ArgumentParser
from torch.utils.tensorboard import SummaryWriter

import random, tqdm, sys, math, gzip

In [4]:
# Used for converting between nats and bits
LOG2E = math.log2(math.e)
TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False)
NUM_CLS = 2

In [7]:
# load the IMDB data
final=True
vocab_size = 50000
batch_size = 4
if final:
    train, test = datasets.IMDB.splits(text_field=TEXT, label_field=LABEL, root='../data/processed/')

    TEXT.build_vocab(train, max_size=vocab_size - 2)
    LABEL.build_vocab(train)

    train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=batch_size)
else:
    tdata, _ = datasets.IMDB.splits(TEXT, LABEL)
    train, test = tdata.split(split_ratio=0.8)

    TEXT.build_vocab(train, max_size=vocab_size - 2) # - 2 to make space for <unk> and <pad>
    LABEL.build_vocab(train)

    train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=batch_size)

print(f'- nr. of training examples {len(train_iter)}')
print(f'- nr. of {"test" if final else "validation"} examples {len(test_iter)}')

- nr. of training examples 6250
- nr. of test examples 6250


In [12]:
for i in train_iter:
    text, label = i.text, i.label
    break

In [25]:
max_length = -1
if max_length < 0:
    mx = max([input.text[0].size(1) for input in train_iter])
    mx = mx * 2
    print(f'- maximum sequence length: {mx}')
else:
    mx = max_length
    print(f'- maximum sequence length: {mx}')

- maximum sequence length: 4940
